In [2]:
import pandas as pd
import mysql.connector

In [3]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_44'     
)
cursor = connection.cursor()

### Bài 1

In [4]:
# Khóa chính: agent_id & task_id

### Bài 2

In [5]:
# Có vi phạm 2NP
# agent_name và llm_model phụ thuộc 1 phần vào agent_id
# task_name và task_type phụ thuộc 1 phần vào task_id

### Bài 3

In [6]:
cursor.execute("""
CREATE TABLE Agent (
    agent_id VARCHAR(10) PRIMARY KEY,
    agent_name VARCHAR(200) NOT NULL, 
    llm_model VARCHAR(200) NOT NULL
);
""")

In [8]:
cursor.execute("""
INSERT INTO Agent (agent_id, agent_name, llm_model) VALUES
('A01', 'Claude Agent', 'Claude 3'),
('A02', 'GPT Assistant', 'GPT-4'),
('A03', 'Gemini Helper', 'Gemini 1.5');
""")

connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE Tasks (
    task_id VARCHAR(10) PRIMARY KEY,
    task_name VARCHAR(200) NOT NULL, 
    task_type VARCHAR(200) NOT NULL
);
""")

In [10]:
cursor.execute("""
INSERT INTO Tasks (task_id, task_name, task_type) VALUES
('T02', 'SQL Generation', 'Code'),
('T03', 'Sentiment Analysis', 'NLP'),
('T01', 'Summarization', 'NLP');
""")

connection.commit()

In [11]:
cursor.execute("""
CREATE TABLE AgentTasks (
    task_id VARCHAR(10) NOT NULL,
    agent_id VARCHAR(10) NOT NULL,
    FOREIGN KEY (task_id) REFERENCES Tasks(task_id),
    FOREIGN KEY (agent_id) REFERENCES Agent(agent_id)
);
""")

In [12]:
cursor.execute("""
INSERT INTO AgentTasks (task_id, agent_id) VALUES
('T01', 'A01'),
('T02', 'A02'),
('T03', 'A01'), 
('T01', 'A03');
""")

connection.commit()

### Bài 4

In [13]:
# Bảng Agent có các cột: agent_id (khóa chính), agent_name, llm_model
# Bảng Tasks có các cột: task_id (khóa chính), task_name, task_type
# Bảng AgentTasks có các cột: task_id (khóa ngoại), agent_id (khóa ngoại)

### Bài 5

In [15]:
query = ("""
SELECT b.task_name, a.agent_name
FROM AgentTasks c
LEFT JOIN Agent a ON c.agent_id = a.agent_id
LEFT JOIN Tasks b ON c.task_id = b.task_id
""")

data_5a = pd.read_sql_query(query, connection)
print(data_5a)

            task_name     agent_name
0       Summarization   Claude Agent
1      SQL Generation  GPT Assistant
2  Sentiment Analysis   Claude Agent
3       Summarization  Gemini Helper


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_9068\1832001570.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5a = pd.read_sql_query(query, connection)


In [16]:
query = ("""
SELECT llm_model
FROM Agent  
WHERE agent_id IN (SELECT agent_id FROM AgentTasks)
""")

data_5b = pd.read_sql_query(query, connection)
print(data_5b)

    llm_model
0    Claude 3
1       GPT-4
2  Gemini 1.5


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_9068\1530377249.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5b = pd.read_sql_query(query, connection)


In [17]:
query = ("""
SELECT task_type, COUNT(task_type) 
FROM Tasks  
GROUP BY task_type
""")

data_5c = pd.read_sql_query(query, connection)
print(data_5c)

  task_type  COUNT(task_type)
0       NLP                 2
1      Code                 1


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_9068\3451656160.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5c = pd.read_sql_query(query, connection)


In [19]:
query = ("""
SELECT agent_name, llm_model
FROM Agent  
WHERE llm_model LIKE 'G%'
""")

data_5d = pd.read_sql_query(query, connection)
print(data_5d)

      agent_name   llm_model
0  GPT Assistant       GPT-4
1  Gemini Helper  Gemini 1.5


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_9068\2888069077.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5d = pd.read_sql_query(query, connection)
